In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import random
import warnings
import statistics as stats
import time
import math
import pandas as pd
from google.colab import files

In [2]:
def plot(digraph):
    
    plt.rcParams["figure.figsize"] = [12, 12]
    plt.rcParams["figure.autolayout"] = True
    
    pos = nx.circular_layout(digraph)
   
    plt.show()
    warnings.filterwarnings("ignore")
    nx.draw(digraph, pos,with_labels=True,node_color='pink')

    

In [3]:
#classic hill climbing : returns the best possible neighbor
def generateBestNeighborClassicHillClimbing(state, initialStateCopy):
    bestState = state
    stateNodes = list(state.nodes())
    random.shuffle(stateNodes) 
    
    for node in stateNodes:
        
        stateCopyForRemoving = state.copy()
        stateCopyForAdding = state.copy()
       
        #removing a node and testing the value of the resulting graph
        if (nbNodesInAtLeastOneCycle(state) > 0):
            stateCopyForRemoving.remove_node(node)
            bestState = bestStateBetween(bestState, stateCopyForRemoving)
        
        #adding a node and testing the value of the resulting graph
        if (initialStateCopy.number_of_nodes() > state.number_of_nodes()):
            initialNodes = list(initialStateCopy.nodes())
            initialEdges = list(initialStateCopy.edges())
            nodeToAdd = random.choice([elem for elem in initialNodes if elem not in stateNodes])
            edgesToAdd = [elem for elem in initialEdges if (elem[0] == nodeToAdd and elem[1] in stateNodes) or (elem[1] == nodeToAdd and elem[0] in stateNodes)]
            stateCopyForAdding.add_edges_from(edgesToAdd)
            bestState = bestStateBetween(bestState, stateCopyForAdding)
   
    return bestState

In [4]:
#first-choice hill climbing : returns the first better neighbor
def generateBestNeighborFirstChoiceHillClimbing(state, initialStateCopy):
    stateNodes = list(state.nodes())
    random.shuffle(stateNodes)
   
    for node in stateNodes:
        
        stateCopyForRemoving = state.copy()
        stateCopyForAdding = state.copy()
       
        #removing a node and testing the value of the resulting graph
        if (nbNodesInAtLeastOneCycle(state) > 0):
            stateCopyForRemoving.remove_node(node)
            betterState = bestStateBetween(state, stateCopyForRemoving)
            if (betterState != state):
                print("\nnode removed :", node)
                return betterState
        
        #adding a node and testing the value of the resulting graph
        if (initialStateCopy.number_of_nodes() > state.number_of_nodes()):
            initialNodes = list(initialStateCopy.nodes())
            initialEdges = list(initialStateCopy.edges())
            nodeToAdd = random.choice([elem for elem in initialNodes if elem not in stateNodes])
            edgesToAdd = [elem for elem in initialEdges if (elem[0] == nodeToAdd and elem[1] in stateNodes) or (elem[1] == nodeToAdd and elem[0] in stateNodes)]
            stateCopyForAdding.add_edges_from(edgesToAdd)
            betterState = bestStateBetween(state, stateCopyForAdding)
            if (betterState != state):
                print("\nnode added :", nodeToAdd)
                return betterState
    
    return state

In [5]:
#returns if a path exists between u and v
def existPath(matriceAdj, u, v):
   
    n = len(matriceAdj) # number of nodes
    file = []
    visites = [False] * n
    file.append(u)
    while file:
        courant = file.pop(0)
        visites[courant] = True
        
        for i in range(n):
            if matriceAdj[courant,i] > 0 and visites[i] == False:
                file.append(i)
                visites[i] = True
            elif matriceAdj[courant,i] > 0 and i == v:
                return True 
    return False

In [6]:
#returns the number of nodes in at least one cycle in the given state
def nbNodesInAtLeastOneCycle(state):
    count = 0
    adjacencyMatrix=nx.adjacency_matrix(state)
    adjacencyMatrix=adjacencyMatrix.todense()
    
   #using existPath
    for node in range(0,len(adjacencyMatrix)):
      if existPath(adjacencyMatrix, node, node):
        count+=1
    return count

In [7]:
#returns the best state between current state and the neighbor state
def bestStateBetween(currentState, neighborState):
    nbNodesInAtLeastOneCycleCurrentState = nbNodesInAtLeastOneCycle(currentState)
    nbNodesInAtLeastOneCycleNeighbor = nbNodesInAtLeastOneCycle(neighborState) 
    
   
    if (nbNodesInAtLeastOneCycleNeighbor < nbNodesInAtLeastOneCycleCurrentState):
        return neighborState
    elif (nbNodesInAtLeastOneCycleNeighbor == nbNodesInAtLeastOneCycleCurrentState and neighborState.number_of_nodes() > currentState.number_of_nodes()):
        return neighborState
    return currentState

In [8]:
def hillClimbing(nbNodes, probaErdosRenyi, plotResults, method=2):
    startTime = time.time()

    nbNodes = nbNodes
    probaErdosRenyi = probaErdosRenyi
    initialState = nx.generators.random_graphs.erdos_renyi_graph(nbNodes,probaErdosRenyi, directed= True)
    initialStateCopy = initialState.copy()


    print("\n\n\nWe start with this initial graph : " , list(initialStateCopy.edges()))


    # hill climbing algorithm
    currentState = initialState
    while (1):
        if(method == 1):
           neighborState = generateBestNeighborClassicHillClimbing(currentState, initialStateCopy)
        elif(method == 2):
           neighborState = generateBestNeighborFirstChoiceHillClimbing(currentState, initialStateCopy)
        bestState = bestStateBetween(currentState, neighborState)
        if (bestState == currentState):
            break;
        currentState = neighborState
        print("\ncurrent state : ",list(currentState.edges()))
      
    print("\n\n-------------------END OF ALGORITHM-------------------\n")
    try:
      print("Cycle in solution ? ",nx.find_cycle(currentState))
    except nx.exception.NetworkXNoCycle as e:
      print("Found no cycle ")

    # we made an acyclic graph out of the initial graph
    # now we must deduce the cycle cutter and display it
    cycleCutter = [elem for elem in list(initialStateCopy.nodes()) if elem not in list(currentState.nodes())]

    finishTime = time.time()
    executionTime = finishTime - startTime

    print("\n\n-------------------CONCLUSION-------------------\n\nThe initial graph had", nbNodes, "nodes and", nbNodesInAtLeastOneCycle(initialStateCopy), "nodes in at least one cycle.")
    print("\nAnd we obtained the following acyclic graph :\n\n" , list(currentState.edges()))
    print("\nThus the cycle-cutter is :", cycleCutter, ", the hill-climbing algorithm removed", len(cycleCutter), "nodes, with execution time =", round(executionTime,2), "seconds.")

    if (plotResults):
      print("\n\n-------------------INITIAL STATE (Fig 1) VS FINAL STATE (Fig 2)-------------------\n")
      plot(initialStateCopy)
      plot(bestState)

    return (len(cycleCutter),executionTime)

In [ ]:
#Testez l’algorithme de Hill Climbing dans cette cellule ***************************************************************************
hillClimbing(50,0.08,True, 2)

In [ ]:
# statistics
'''
nbOfNodesToConsider = [10,30,70,100]
probasErdosRenyiToConsider = [0.02,0.04,0.06,0.08,0.1,0.2,0.3,0.4,0.5]
methodsToConsider = [1,2]
sampleSize = 15

for nbNode in nbOfNodesToConsider: 
  
  nodeListDf = []
  probaListDf = []
  methodListDf = []
  meanCycleCutterListDf = []
  standardDeviationCycleCutterListDf = []
  meanExecutionTimeListDf = []
  standarDeviationExecutionTimeListDf = []

  for methodNb in methodsToConsider:
    for proba in probasErdosRenyiToConsider:
        nbNodes = nbNode
        probaErdosRenyi = proba
        method = methodNb # method : 1 for classic, 2 for first-choice
        # plotResults = True only if trial for one graph, not in a loop for several graphs

        resultList = [0] * sampleSize
        executionTimeList = [0] * sampleSize
        #generating 50 graphs with these parameters
        for i in range(sampleSize):
          result = hillClimbing(nbNodes,probaErdosRenyi,False, method)
          resultList[i] = result[0]
          executionTimeList[i] = result[1]

        averageExecutionTime = round((sum(executionTimeList)/len(executionTimeList)),2)
        mean = round(sum(resultList)/len(resultList),2)
        standardDeviation = round(math.sqrt(stats.pvariance(resultList)),2)
        standardDeviationExecutionTime = round(math.sqrt(stats.pvariance(executionTimeList)),2)

        print("\n\n-------------------MEAN & STANDARD DEVIATION-------------------\n")
        print("The initial graph had", nbNodes, "nodes and probability of Erdos-Renyi", probaErdosRenyi,".")
        print("\nIn average, the cycle-cutter contains :", mean, "nodes, with a standard deviation of :", standardDeviation,".")
        print("\nIn average, the execution time of the hill climbing algorithm with the above parameters is :", averageExecutionTime, "seconds, with a standard deviation of :",standardDeviationExecutionTime,".")

        #completing lists for the dataframe
        nodeListDf.append(nbNode)
        probaListDf.append(proba)
        if (methodNb == 1):
          methodListDf.append("classique")
        if (methodNb == 2):
          methodListDf.append("first choice")
        meanCycleCutterListDf.append(mean)
        standardDeviationCycleCutterListDf.append(standardDeviation)
        meanExecutionTimeListDf.append(averageExecutionTime)
        standarDeviationExecutionTimeListDf.append(standardDeviationExecutionTime)

  #creating a report for each number of nodes in graph
  results = {"nombre de sommets" : nodeListDf, "proba Erdos-Renyi" : probaListDf, "méthode (classique ou first choice)" : methodListDf, "moyenne taille coupe cycle" : meanCycleCutterListDf, "écart-type taille coupe cycle" : standardDeviationCycleCutterListDf, "moyenne temps exécution" : meanExecutionTimeListDf, "écart-type temps exécution" : standarDeviationExecutionTimeListDf,}
  results_df = pd.DataFrame(results)
  moncsv = results_df.to_csv("results_" + str(nbNode) +".csv", sep=";")
  files.download("results_" + str(nbNode) +".csv") 

  '''